### Word In Visual Genome

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

file_ext = 1000

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-1-Word In Visual Genome"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df, search_list, target_column, sample_num)\n
    df is dataframe and target_column is its column for external searching_list\n
    sample_num for take sample.
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).head(sample_num)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len()).head(sample_num)               
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_dataframe_all(df, search_list, target_column):
    '''
    word_group_dataframe(df, search_list, target_column)\n
    df is dataframe and target_column is its column for external searching_list\n
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len())
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)        
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

#### Visual Genome Data

In [6]:
df_genome_objects = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-0-Visual Genome Process/Visual_Genome_Objects_Analysis.csv")
df_genome_objects.drop("image_url", inplace=True, axis=1)
df_genome_objects

,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,1058549,trees,1,tree.n.01,NaN,557,799,0,0
1,1058534,sidewalk,1,sidewalk.n.01,5046,290,722,78,308
2,1058508,building,1,building.n.01,NaN,538,222,1,0
3,1058539,street,1,street.n.01,3798578,258,359,439,283
4,1058543,wall,1,wall.n.01,NaN,535,135,0,1
...,...,...,...,...,...,...,...,...,...
2516934,3506171,number,2417997,numeral.n.01,NaN,24,28,188,228
2516935,3786799,plate,2417997,plate.n.04,NaN,54,89,238,233
2516936,3103226,road,2417997,road.n.01,NaN,83,90,312,247
2516937,3094589,road,2417997,road.n.01,NaN,121,495,2,208


In [7]:
df_genome_objects["names"].nunique()

82825

In [8]:
df_genome_attributes = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-0-Visual Genome Process/Visual_Genome_Attributes_Analysis.csv")
df_genome_attributes

,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,1058498,clock,1,"green , tall",clock.n.01,339,79,421,91
1,5046,street,1,sidewalk,street.n.01,262,714,77,328
2,5045,shade,1,NaN,shade.n.01,192,274,119,338
3,1058529,man,1,NaN,man.n.01,262,60,238,249
4,5048,sneakers,1,grey,gym_shoe.n.01,26,52,243,489
...,...,...,...,...,...,...,...,...,...
3802369,3422177,sky,2417997,"crystal clear , blue",sky.n.01,170,497,1,2
3802370,3103226,road,2417997,NaN,road.n.01,83,90,312,247
3802371,2712437,doors,2417997,NaN,door.n.01,144,50,98,127
3802372,3140197,bus,2417997,green,bus.n.01,155,225,29,109


In [9]:
df_genome_relationships = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-0-Visual Genome Process/Visual_Genome_Relationships_Analysis.csv")
df_genome_relationships

,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,1,sidewalk,1058534,sidewalk.n.01,5046,290,722,78,308,15927,along.r.01,shade,5045,shade.n.01,192,274,119,338,on
1,1,shoes,1058525,shoe.n.01,5048,28,48,388,485,15928,wear.v.01,man,1058529,man.n.01,262,60,238,249,wears
2,1,NaN,5050,headlight.n.01,NaN,15,23,514,366,15929,have.v.01,car,5049,car.n.01,98,74,479,315,has
3,1,NaN,1058508,building.n.01,NaN,536,218,1,2,15930,along.r.01,sign,1058507,sign.n.02,182,88,118,13,on
4,1,NaN,1058534,sidewalk.n.01,NaN,266,722,77,331,15931,along.r.01,tree trunk,5055,trunk.n.01,327,87,622,234,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316099,2417997,NaN,3103225,bus.n.01,NaN,193,289,14,110,4245033,along.r.01,plate,3786799,plate.n.04,54,89,238,233,on
2316100,2417997,NaN,3786800,desert.n.01,NaN,119,157,301,211,4245034,in.r.01,NaN,3103225,bus.n.01,182,287,14,110,in
2316101,2417997,NaN,3103226,road.n.01,NaN,83,90,312,247,3722727,along.r.01,bus,3103225,bus.n.01,193,289,14,110,on
2316102,2417997,NaN,2712437,door.n.01,NaN,144,50,98,127,3880229,NaN,bus,3103225,bus.n.01,193,289,14,110,with


In [10]:
df_genome_question_answers = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-0-Visual Genome Process/Visual_Genome_Question_Answers_Analysis.csv")
df_genome_question_answers

,image_id,questions,que_and_anw_id,answers
0,2,what color is the car,986934,red
1,2,what color are the trees,986935,green
2,2,what has lots of glass windows,986936,the building
3,2,what is the guy doing,986937,walking
4,2,what is the man doing,986938,crossing the street
...,...,...,...,...
1445226,2417995,when was the photo taken,955110,day time
1445227,2417995,why is it so bright,955111,due to natural sunlight
1445228,2417995,how many trains are there,955112,one
1445229,2417995,what is the train made of,955113,steel


In [11]:
df_genome_region_descriptions = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-0-Visual Genome Process/Visual_Genome_Region_Descriptions_Analysis.csv")
df_genome_region_descriptions

,image_id,phrases,region_id,height,width,x_koor,y_koor
0,1,the clock is green in colour,1382,139,82,421,57
1,1,shade is along the street,1383,109,182,194,372
2,1,man is wearing sneakers,1384,30,61,241,491
3,1,cars headlights are off,1385,36,36,617,377
4,1,bikes are parked at the far edge,1386,49,41,322,298
...,...,...,...,...,...,...,...
5408684,2417997,a green school bus traveling down a road,5516956,181,303,0,111
5408685,2417997,numbers on the front of a bus,5516957,15,23,191,231
5408686,2417997,a yellow license plate on a bus,5516958,13,17,259,247
5408687,2417997,a country road between two mountains,5516959,112,317,180,218


#### Word Search In Genome Data

In [12]:
#df_word_raw = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Result/2-Word Select Process/{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
#df_word_raw

In [13]:
#word_list = ['an','anlama','doğum','gidiyor','günün','istediğim','istediğin','istediğiniz','kutlu','olmuş','soru','söylemek','zamanki','zor','şuna']

In [14]:
#df_word_raw = df_word_select[df_word_select["word"].isin(word_list)]
#df_word_raw.reset_index(drop=True, inplace=True)
#df_word_raw.drop(["frequency","stem"], inplace=True, axis=1)
#df_word_raw

In [15]:
df_word_raw = pd.read_excel(f"Turkish_{file_ext}_Process.xlsx")
df_word_raw

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency
0,NUM,NaN,bir,bir,bir,a,a,18835735
1,PRON,NaN,bu,bu,bu,this,this,11062659
2,PRON,Q,ne,ne,ne,what,what,8025880
3,CCONJ,NaN,ve,ve,ve,and,and,7766036
4,ADP,NaN,için,için,için,for,for,5484109
...,...,...,...,...,...,...,...,...
995,ADJ,NaN,resmi,resmi,resmi,formal,formal,68287
996,VERB,NaN,veriyor,ver,ver,giving,give,68163
997,NOUN,NaN,okul,okul,oku,school,school,68160
998,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124


In [16]:
df_word_raw_word_trans_list = list(set(df_word_raw["word_en_translate"]))
df_word_raw_lemma_trans_list = list(set(df_word_raw["lemma_en_translate"]))

In [17]:
len(df_word_raw_word_trans_list)

818

In [18]:
len(df_word_raw_lemma_trans_list)

486

In [19]:
df_objects_search_word_result = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 50)
df_objects_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_objects_search_word_result

,word_en_translate,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,do,3617319,do,2317891,bash.n.02,3497687,21,14,376,263
1,do,999220,do,2319741,bash.n.02,NaN,66,46,251,144
2,do,3379494,do,2320586,bash.n.02,NaN,21,28,219,90
3,do,3918774,do,2342575,NaN,"3459293 , 3411544",23,36,207,293
4,do,328380,do,2405891,bash.n.02,"328381 , 328343 , 328392",12,11,227,158
...,...,...,...,...,...,...,...,...,...,...
10483,cut,358312,cut,2411671,cut.n.01,NaN,13,58,196,132
10484,cut,295583,cut,2414418,cut.n.01,NaN,40,28,294,105
10485,cut,232707,cut,2409824,cut.n.01,NaN,81,25,253,286
10486,cut,4100327,cut,2323488,NaN,NaN,46,53,149,29


In [20]:
df_objects_search_word_result.to_csv(f"Visual_Genome_Objects_{file_ext}_Word_Result.csv", index=False)

In [21]:
df_objects_search_lemma_result = word_group_dataframe(df_genome_objects, df_word_raw_lemma_trans_list, "names", 50)
df_objects_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_objects_search_lemma_result

,lemma_en_translate,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,do,3617319,do,2317891,bash.n.02,3497687,21,14,376,263
1,do,999220,do,2319741,bash.n.02,NaN,66,46,251,144
2,do,3379494,do,2320586,bash.n.02,NaN,21,28,219,90
3,do,3918774,do,2342575,NaN,"3459293 , 3411544",23,36,207,293
4,do,328380,do,2405891,bash.n.02,"328381 , 328343 , 328392",12,11,227,158
...,...,...,...,...,...,...,...,...,...,...
10018,cut,358312,cut,2411671,cut.n.01,NaN,13,58,196,132
10019,cut,295583,cut,2414418,cut.n.01,NaN,40,28,294,105
10020,cut,232707,cut,2409824,cut.n.01,NaN,81,25,253,286
10021,cut,4100327,cut,2323488,NaN,NaN,46,53,149,29


In [22]:
df_objects_search_lemma_result.to_csv(f"Visual_Genome_Objects_{file_ext}_Lemma_Result.csv", index=False)

In [23]:
df_attributes_search_word_result = word_group_dataframe(df_genome_attributes, df_word_raw_word_trans_list, "attributes", 50)
df_attributes_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_attributes_search_word_result

,word_en_translate,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,do,4494249,word,2389438,do,NaN,28,45,298,117
1,do,1213447,sticker,2394195,do not,gummed_label.n.01,60,49,197,43
2,do,1710214,walk signal,2361923,do not,NaN,20,10,411,171
3,do,4047458,signal,2338739,do no walk,NaN,97,27,203,249
4,do,1779577,aircraft,2349775,do landing,aircraft.n.01,12,32,346,207
...,...,...,...,...,...,...,...,...,...,...
12933,cut,639509,tomato,2364716,cut,tomato.n.01,31,37,324,63
12934,cut,2243572,man,2364590,cut,man.n.01,162,143,140,84
12935,cut,4061498,boy,2317842,cut,NaN,189,137,76,72
12936,cut,3737505,slice,2364586,cut,slice.n.01,66,159,1,123


In [24]:
df_attributes_search_word_result.to_csv(f"Visual_Genome_Attributes_{file_ext}_Word_Result.csv", index=False)

In [25]:
df_attributes_search_lemma_result = word_group_dataframe(df_genome_attributes, df_word_raw_lemma_trans_list, "attributes", 50)
df_attributes_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_attributes_search_lemma_result

,lemma_en_translate,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,do,4494249,word,2389438,do,NaN,28,45,298,117
1,do,1213447,sticker,2394195,do not,gummed_label.n.01,60,49,197,43
2,do,1710214,walk signal,2361923,do not,NaN,20,10,411,171
3,do,4047458,signal,2338739,do no walk,NaN,97,27,203,249
4,do,1779577,aircraft,2349775,do landing,aircraft.n.01,12,32,346,207
...,...,...,...,...,...,...,...,...,...,...
11272,related,912084,projection,2344910,business related,projection.n.01,106,137,186,0
11273,related,2209408,sign,2377350,bike lane related,sign.n.02,54,56,131,439
11274,related,2313354,documents,2356589,"necessary , travel related",document.n.01,125,47,346,276
11275,related,1179791,"giraffe , giraffes , animals",2398447,"spotted , brown , tan , walking , related , happy",giraffe.n.01,409,247,50,34


In [26]:
df_attributes_search_lemma_result.to_csv(f"Visual_Genome_Attributes_{file_ext}_Lemma_Result.csv", index=False)

In [27]:
df_relationships_search_word_result = word_group_dataframe(df_genome_relationships, df_word_raw_word_trans_list, "sub_name", 50)
df_relationships_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_relationships_search_word_result

,word_en_translate,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,do,2405891,do,328380,bash.n.02,"328381 , 328343 , 328392",12,11,227,158,511767,not.r.01,do,328380,bash.n.02,12,11,227,158,not
1,do,2402486,NaN,1135469,sign.n.02,NaN,79,108,123,42,3274620,along.r.01,do,1135472,bash.n.02,15,21,130,68,on
2,do,2389064,NaN,507073,sofa.n.01,NaN,176,415,37,109,1203350,put.v.01,do,507070,bash.n.02,86,180,262,166,laying
3,do,2388016,NaN,1271100,NaN,NaN,38,19,389,191,4293050,not.r.01,do,3829684,bash.n.02,64,86,196,152,not
4,do,2320586,NaN,3379495,NaN,NaN,20,35,258,90,3862722,in.r.01,do,3379494,bash.n.02,25,29,215,87,in front of
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8520,cut,2348390,NaN,882550,dog.n.01,NaN,289,353,145,34,2560353,NaN,cut end,882556,NaN,168,129,148,147,of
8521,cut,2407875,NaN,272075,table.n.02,NaN,102,463,1,271,321147,along.r.01,cut out,272089,NaN,20,30,100,285,on
8522,cut,2336525,NaN,4590747,NaN,NaN,81,92,320,15,4796921,NaN,cut out,4590738,NaN,241,313,95,6,has
8523,cut,2334797,NaN,4643735,NaN,NaN,156,299,62,214,4818683,NaN,cut out,4643749,NaN,133,143,204,93,above


In [28]:
df_relationships_search_word_result.to_csv(f"Visual_Genome_Relationships_{file_ext}_Word_Result.csv", index=False)

In [29]:
df_relationships_search_lemma_result = word_group_dataframe(df_genome_relationships, df_word_raw_lemma_trans_list, "sub_name", 50)
df_relationships_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_relationships_search_lemma_result

,lemma_en_translate,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,do,2405891,do,328380,bash.n.02,"328381 , 328343 , 328392",12,11,227,158,511767,not.r.01,do,328380,bash.n.02,12,11,227,158,not
1,do,2402486,NaN,1135469,sign.n.02,NaN,79,108,123,42,3274620,along.r.01,do,1135472,bash.n.02,15,21,130,68,on
2,do,2389064,NaN,507073,sofa.n.01,NaN,176,415,37,109,1203350,put.v.01,do,507070,bash.n.02,86,180,262,166,laying
3,do,2388016,NaN,1271100,NaN,NaN,38,19,389,191,4293050,not.r.01,do,3829684,bash.n.02,64,86,196,152,not
4,do,2320586,NaN,3379495,NaN,NaN,20,35,258,90,3862722,in.r.01,do,3379494,bash.n.02,25,29,215,87,in front of
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8353,cut,2348390,NaN,882550,dog.n.01,NaN,289,353,145,34,2560353,NaN,cut end,882556,NaN,168,129,148,147,of
8354,cut,2407875,NaN,272075,table.n.02,NaN,102,463,1,271,321147,along.r.01,cut out,272089,NaN,20,30,100,285,on
8355,cut,2336525,NaN,4590747,NaN,NaN,81,92,320,15,4796921,NaN,cut out,4590738,NaN,241,313,95,6,has
8356,cut,2334797,NaN,4643735,NaN,NaN,156,299,62,214,4818683,NaN,cut out,4643749,NaN,133,143,204,93,above


In [30]:
df_relationships_search_lemma_result.to_csv(f"Visual_Genome_Relationships_{file_ext}_Lemma_Result.csv", index=False)

In [31]:
df_question_answers_search_word_result = word_group_dataframe(df_genome_question_answers, df_word_raw_word_trans_list, "questions", 50)
df_question_answers_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_question_answers_search_word_result

,word_en_translate,image_id,questions,que_and_anw_id,answers
0,do,2389512,why do this,547971,for fun
1,do,2347713,who do you see,581362,one person
2,do,2408495,what do you se,1061211,mane of giraffe
3,do,2360866,why do you buy,762733,get cuddly heart
4,do,2401493,who do you see,538380,nobody
...,...,...,...,...,...
17987,cut,2346722,what is being cut,262225,fruit
17988,cut,2398301,what is being cut,1462778,desert
17989,cut,2398118,how is banana cut,1398551,in round pieces
17990,you want,2393378,why wouldn t you want to sit on the benches,352929,they re covered with snow


In [32]:
df_question_answers_search_word_result.to_csv(f"Visual_Genome_Question_Answers_{file_ext}_Word_Result.csv", index=False)

In [33]:
df_question_answers_search_word_result2 = word_group_dataframe(df_genome_question_answers, df_word_raw_word_trans_list, "answers", 50)
df_question_answers_search_word_result2.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_question_answers_search_word_result2

,word_en_translate,image_id,questions,que_and_anw_id,answers
0,do,2403084,what is the first word on the sign,1275002,do
1,do,2367872,what is the woman s hair doing,1790330,up do
2,do,107975,what is the woman s book about,996929,su do ku
3,do,2413935,how do you know where he is surfing,144907,you do not
4,do,2365231,what does the man s shirt say,1737588,do the dew
...,...,...,...,...,...
17206,cut,2360029,why is the knife used,1633213,to cut
17207,cut,2399822,how are the cherry tomatoes,1532086,cut up
17208,cut,2406715,why are the scissors there,202668,to cut
17209,cut,2372642,how is the fruit served,1889331,cut up


In [34]:
df_question_answers_search_word_result2.to_csv(f"Visual_Genome_Question_Answers_{file_ext}_Word_Result2.csv", index=False)

In [35]:
df_question_answers_search_lemma_result = word_group_dataframe(df_genome_question_answers, df_word_raw_lemma_trans_list, "questions", 50)
df_question_answers_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_question_answers_search_lemma_result

,lemma_en_translate,image_id,questions,que_and_anw_id,answers
0,do,2389512,why do this,547971,for fun
1,do,2347713,who do you see,581362,one person
2,do,2408495,what do you se,1061211,mane of giraffe
3,do,2360866,why do you buy,762733,get cuddly heart
4,do,2401493,who do you see,538380,nobody
...,...,...,...,...,...
14207,related,2404860,what barnyard animal are these animals related to,1148033,horses
14208,related,2365858,what business related contraption is in this i...,1750004,fax machine
14209,related,2402476,what math related symbol is on the game contro...,1263154,plus
14210,related,2401283,what transportation related area is visible in...,1240692,roadway


In [36]:
df_question_answers_search_lemma_result.to_csv(f"Visual_Genome_Question_Answers_{file_ext}_Lemma_Result.csv", index=False)

In [37]:
df_question_answers_search_lemma_result2 = word_group_dataframe(df_genome_question_answers, df_word_raw_lemma_trans_list, "answers", 50)
df_question_answers_search_lemma_result2.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_question_answers_search_lemma_result2

,lemma_en_translate,image_id,questions,que_and_anw_id,answers
0,do,2403084,what is the first word on the sign,1275002,do
1,do,2367872,what is the woman s hair doing,1790330,up do
2,do,107975,what is the woman s book about,996929,su do ku
3,do,2413935,how do you know where he is surfing,144907,you do not
4,do,2365231,what does the man s shirt say,1737588,do the dew
...,...,...,...,...,...
13981,related,2366245,why are the zebras near each other,1757754,they are related
13982,related,2392709,what kind of stickers are these,1435705,electronics related
13983,related,2388188,why do these women resemble each other,1536776,they might be related
13984,related,2345416,why does it look like a horse or donkey,454146,they are closely related


In [38]:
df_question_answers_search_lemma_result2.to_csv(f"Visual_Genome_Question_Answers_{file_ext}_Lemma_Result2.csv", index=False)

In [39]:
df_region_descriptions_search_word_result = word_group_dataframe(df_genome_region_descriptions, df_word_raw_word_trans_list, "phrases", 50)
df_region_descriptions_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_region_descriptions_search_word_result

,word_en_translate,image_id,phrases,region_id,height,width,x_koor,y_koor
0,do,2388631,a glazed do nut,1302229,70,85,59,109
1,do,2336163,a do co trailer,3651536,47,84,127,233
2,do,2318722,do not turn sign,4438879,36,25,432,205
3,do,2318722,do not turn sign,4438882,27,21,156,238
4,do,2391751,do not turn sign,1151902,28,31,141,280
...,...,...,...,...,...,...,...,...
22735,cut,2321602,a cut up tree,4703157,46,30,323,196
22736,cut,2395063,a cut of meat,993646,126,149,225,152
22737,cut,2395063,a cut of meat,993645,142,112,182,206
22738,cut,2383857,short cut hair,1929496,42,51,127,5


In [40]:
df_region_descriptions_search_word_result.to_csv(f"Visual_Genome_Region_Descriptions_{file_ext}_Word_Result.csv", index=False)

In [41]:
df_region_descriptions_search_lemma_result = word_group_dataframe(df_genome_region_descriptions, df_word_raw_lemma_trans_list, "phrases", 50)
df_region_descriptions_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_region_descriptions_search_lemma_result

,lemma_en_translate,image_id,phrases,region_id,height,width,x_koor,y_koor
0,do,2388631,a glazed do nut,1302229,70,85,59,109
1,do,2336163,a do co trailer,3651536,47,84,127,233
2,do,2318722,do not turn sign,4438879,36,25,432,205
3,do,2318722,do not turn sign,4438882,27,21,156,238
4,do,2391751,do not turn sign,1151902,28,31,141,280
...,...,...,...,...,...,...,...,...
18140,related,2355690,illuminated signs indicate space related images,2873732,66,58,11,110
18141,related,2377614,four people at an alcohol related establishment,1630196,290,377,18,74
18142,related,2404120,grouping of probably related people on front r...,561584,289,438,30,33
18143,related,2343521,badges are usually a good indication of a work...,3454219,52,17,395,112


In [42]:
df_region_descriptions_search_lemma_result.to_csv(f"Visual_Genome_Region_Descriptions_{file_ext}_Lemma_Result.csv", index=False)

#### Copy Move And Delete

In [43]:
output_file = glob.glob(f"Visual_Genome_*_Result*.csv")
output_file

['Visual_Genome_Objects_1000_Word_Result.csv',
 'Visual_Genome_Objects_1000_Lemma_Result.csv',
 'Visual_Genome_Attributes_1000_Word_Result.csv',
 'Visual_Genome_Attributes_1000_Lemma_Result.csv',
 'Visual_Genome_Relationships_1000_Word_Result.csv',
 'Visual_Genome_Relationships_1000_Lemma_Result.csv',
 'Visual_Genome_Question_Answers_1000_Word_Result.csv',
 'Visual_Genome_Question_Answers_1000_Word_Result2.csv',
 'Visual_Genome_Question_Answers_1000_Lemma_Result.csv',
 'Visual_Genome_Question_Answers_1000_Lemma_Result2.csv',
 'Visual_Genome_Region_Descriptions_1000_Word_Result.csv',
 'Visual_Genome_Region_Descriptions_1000_Lemma_Result.csv']

In [44]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [45]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass